<a href="https://colab.research.google.com/github/robbybrodie/time_as_computational_cost/blob/main/experiments/notebooks/dof_mechanisms_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Degrees of Freedom Mechanisms Explorer

This notebook explores different mechanisms for how degrees of freedom (DoF) in a vibrating network substrate change with motion and curvature, testing the theoretical framework from "Capacity-Limited Unification of SR and GR".

## Key Questions
1. Which mechanism(s) naturally produce the observed B(N) behavior?
2. Can we derive time dilation without curve-fitting to GR?
3. What are the falsifiable predictions that differ from GR?
4. How do thermodynamic and string theory interpretations connect?

## Setup and Imports

In [ ]:
# Install required packages if running in Colab
import sys
if 'google.colab' in sys.modules:
    !pip install numpy matplotlib scipy

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import ipywidgets as widgets
from IPython.display import display, HTML
from typing import Tuple, Dict, List, Optional, Callable
from dataclasses import dataclass
from abc import ABC, abstractmethod

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Setup complete!")

## Core Framework

First, let's implement the basic substrate framework and DoF mechanism interface.

In [ ]:
@dataclass
class VoxelState:
    """State of a single voxel in the substrate network."""
    position: np.ndarray  # 3D position
    capacity: float       # Total capacity budget
    used_capacity: float  # Currently used capacity
    connections: List[int]  # Indices of connected voxels
    modes: np.ndarray     # Available vibrational modes
    phase: np.ndarray     # Phase information for each mode
    
    @property
    def available_capacity(self) -> float:
        return self.capacity - self.used_capacity
    
    @property
    def dof_count(self) -> int:
        """Number of accessible degrees of freedom."""
        return len(self.modes)

class SubstrateNetwork:
    """Base class for the discrete substrate network."""
    
    def __init__(self, 
                 grid_size: Tuple[int, int, int] = (10, 10, 10),
                 voxel_capacity: float = 1.0,
                 planck_length: float = 1.0):
        self.grid_size = grid_size
        self.voxel_capacity = voxel_capacity
        self.planck_length = planck_length
        self.total_voxels = np.prod(grid_size)
        self.c = 1.0  # Speed of light (natural units)
        
    def compute_smear(self, velocity: np.ndarray) -> float:
        """Compute kinematic smear parameter ŝ."""
        v_mag = np.linalg.norm(velocity)
        return min(v_mag / self.c, 0.999)
    
    def compute_load(self, mass_density: np.ndarray) -> float:
        """Compute gravitational load parameter λ̂."""
        total_mass = np.sum(mass_density)
        return min(total_mass / (self.total_voxels * self.voxel_capacity), 0.999)
    
    def capacity_constraint_satisfied(self, s_hat: float, lambda_hat: float) -> bool:
        """Check if capacity constraint ŝ² + λ̂² ≤ 1 is satisfied."""
        return s_hat**2 + lambda_hat**2 <= 1.0
    
    def compute_gamma(self, s_hat: float, lambda_hat: float, p: float = 1.0) -> float:
        """Compute the dilation factor Γ(ŝ, λ̂)."""
        if not self.capacity_constraint_satisfied(s_hat, lambda_hat):
            raise ValueError("Capacity constraint violated")
            
        sr_factor = 1.0 / np.sqrt(1 - s_hat**2)
        N = 1.0 - lambda_hat  # Lapse function
        gr_factor = 1.0 / (N**p)
        
        return sr_factor * gr_factor
    
    def proper_time_rate(self, s_hat: float, lambda_hat: float, p: float = 1.0) -> float:
        """Compute dτ/dt = Γ⁻¹."""
        return 1.0 / self.compute_gamma(s_hat, lambda_hat, p)

class DoFMechanism(ABC):
    """Abstract base class for degrees of freedom mechanisms."""
    
    @abstractmethod
    def compute_dof_reduction(self, 
                            substrate: SubstrateNetwork,
                            s_hat: float, 
                            lambda_hat: float) -> float:
        """Compute the reduction in degrees of freedom."""
        pass
    
    @abstractmethod
    def get_signature(self) -> Dict[str, str]:
        """Return the observational signatures of this mechanism."""
        pass

print("Framework classes defined!")

## DoF Mechanisms

Now let's implement the key mechanisms discussed in the GPT-5 conversation.

In [ ]:
class CausalDiamondThrottling(DoFMechanism):
    """DoF reduction via causal diamond volume throttling."""
    
    def __init__(self, 
                 base_diamond_radius: float = 1.0,
                 motion_compression_factor: float = 1.0,
                 curvature_compression_factor: float = 1.0):
        self.base_radius = base_diamond_radius
        self.motion_factor = motion_compression_factor
        self.curvature_factor = curvature_compression_factor
    
    def compute_diamond_volume(self, s_hat: float, lambda_hat: float) -> float:
        """Compute the accessible volume of the causal diamond."""
        motion_compression = 1.0 - self.motion_factor * s_hat**2
        curvature_compression = 1.0 - self.curvature_factor * lambda_hat
        relative_volume = motion_compression * curvature_compression
        return max(relative_volume, 0.01)
    
    def compute_dof_reduction(self, substrate, s_hat: float, lambda_hat: float) -> float:
        diamond_volume = self.compute_diamond_volume(s_hat, lambda_hat)
        dof_fraction = diamond_volume**(2/3)  # Intermediate scaling
        return dof_fraction
    
    def get_signature(self) -> Dict[str, str]:
        return {
            "time_dilation": "Scales with accessible causal diamond volume",
            "cross_terms": "Specific boost × redshift interactions",
            "anisotropy": "Directional effects along boost axis",
            "dispersion": "Minimal - affects all frequencies equally"
        }

print("Causal Diamond Throttling mechanism defined!")

In [ ]:
class TensionInducedBandgaps(DoFMechanism):
    """DoF reduction via tension-induced frequency bandgaps."""
    
    def __init__(self, 
                 base_frequency_range: Tuple[float, float] = (0.1, 10.0),
                 tension_coupling_strength: float = 1.0,
                 bandgap_width_factor: float = 0.5):
        self.freq_min, self.freq_max = base_frequency_range
        self.tension_coupling = tension_coupling_strength
        self.bandgap_factor = bandgap_width_factor
        self.n_modes = 100
        self.base_frequencies = np.linspace(self.freq_min, self.freq_max, self.n_modes)
    
    def compute_bandgap_structure(self, s_hat: float, lambda_hat: float) -> np.ndarray:
        """Compute which frequencies are blocked by bandgaps."""
        # Combined tension from motion and curvature
        tension_magnitude = self.tension_coupling * (lambda_hat + s_hat**2)
        
        # Bandgap opens around characteristic frequency
        gap_center = self.freq_min + tension_magnitude * (self.freq_max - self.freq_min)
        gap_width = self.bandgap_factor * tension_magnitude * (self.freq_max - self.freq_min)
        
        # Determine accessible frequencies
        accessible = np.ones(self.n_modes, dtype=bool)
        gap_mask = (np.abs(self.base_frequencies - gap_center) < gap_width / 2)
        accessible[gap_mask] = False
        
        # High-frequency cutoff from strong tension
        if tension_magnitude > 0.5:
            cutoff_freq = self.freq_max * (1 - tension_magnitude)
            high_freq_mask = self.base_frequencies > cutoff_freq
            accessible[high_freq_mask] = False
        
        return accessible
    
    def compute_dof_reduction(self, substrate, s_hat: float, lambda_hat: float) -> float:
        accessible_modes = self.compute_bandgap_structure(s_hat, lambda_hat)
        dof_fraction = np.sum(accessible_modes) / len(accessible_modes)
        return dof_fraction
    
    def get_signature(self) -> Dict[str, str]:
        return {
            "time_dilation": "Frequency-dependent due to selective mode blocking",
            "dispersion": "Strong - different frequencies experience different delays",
            "anisotropy": "Directional bandgaps along motion axis",
            "spectral_distortion": "Characteristic gaps in frequency spectrum"
        }

print("Tension-Induced Bandgaps mechanism defined!")

In [ ]:
class ModeCrowdingRedshift(DoFMechanism):
    """DoF reduction via redshift sliding modes out of resolvable window."""
    
    def __init__(self, 
                 readout_window: Tuple[float, float] = (1.0, 5.0),
                 redshift_coupling: float = 1.0):
        self.window_min, self.window_max = readout_window
        self.redshift_coupling = redshift_coupling
        self.n_modes = 100
        self.base_frequencies = np.linspace(0.1, 10.0, self.n_modes)
    
    def compute_redshift_factor(self, s_hat: float, lambda_hat: float) -> float:
        """Compute effective redshift from motion and curvature."""
        # Motion contributes via time dilation
        motion_redshift = s_hat**2 / (2 * (1 - s_hat**2))
        
        # Curvature contributes via gravitational redshift
        curvature_redshift = lambda_hat / (1 - lambda_hat)
        
        total_redshift = self.redshift_coupling * (motion_redshift + curvature_redshift)
        return total_redshift
    
    def compute_dof_reduction(self, substrate, s_hat: float, lambda_hat: float) -> float:
        redshift = self.compute_redshift_factor(s_hat, lambda_hat)
        
        # Redshift moves frequencies downward
        redshifted_frequencies = self.base_frequencies / (1 + redshift)
        
        # Count how many remain in the readout window
        in_window = ((redshifted_frequencies >= self.window_min) & 
                    (redshifted_frequencies <= self.window_max))
        
        dof_fraction = np.sum(in_window) / len(in_window)
        return dof_fraction
    
    def get_signature(self) -> Dict[str, str]:
        return {
            "time_dilation": "Universal mapping between lapse N and in-band mode fraction",
            "spectral_distortion": "Clean spectral distortions of clocks/signals",
            "frequency_dependence": "Systematic redshift of all frequencies",
            "dispersion": "Moderate - affects frequency bands differently"
        }

print("Mode Crowding via Redshift mechanism defined!")

## Interactive Mechanism Explorer

Let's create an interactive widget to explore how different mechanisms behave.

In [ ]:
def create_mechanism_explorer():
    """Create interactive widgets for exploring DoF mechanisms."""
    
    # Initialize mechanisms
    mechanisms = {
        "Causal Diamond": CausalDiamondThrottling(),
        "Tension Bandgaps": TensionInducedBandgaps(),
        "Mode Crowding": ModeCrowdingRedshift()
    }
    
    substrate = SubstrateNetwork()
    
    # Create widgets
    mechanism_dropdown = widgets.Dropdown(
        options=list(mechanisms.keys()),
        value="Causal Diamond",
        description="Mechanism:"
    )
    
    s_hat_slider = widgets.FloatSlider(
        value=0.3,
        min=0.0,
        max=0.95,
        step=0.05,
        description="Smear ŝ:"
    )
    
    lambda_hat_slider = widgets.FloatSlider(
        value=0.3,
        min=0.0,
        max=0.95,
        step=0.05,
        description="Load λ̂:"
    )
    
    p_slider = widgets.FloatSlider(
        value=1.0,
        min=0.5,
        max=2.0,
        step=0.1,
        description="Exponent p:"
    )
    
    def update_plot(mechanism_name, s_hat, lambda_hat, p):
        """Update the comparison plot."""
        mechanism = mechanisms[mechanism_name]
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. DoF surface plot
        ax1 = axes[0, 0]
        s_range = np.linspace(0, 0.9, 30)
        l_range = np.linspace(0, 0.9, 30)
        S, L = np.meshgrid(s_range, l_range)
        DoF = np.zeros_like(S)
        
        for i in range(len(s_range)):
            for j in range(len(l_range)):
                s, l = S[i,j], L[i,j]
                if substrate.capacity_constraint_satisfied(s, l):
                    DoF[i,j] = mechanism.compute_dof_reduction(substrate, s, l)
                else:
                    DoF[i,j] = np.nan
        
        contour1 = ax1.contourf(S, L, DoF, levels=20, cmap='viridis')
        ax1.contour(S, L, S**2 + L**2, levels=[1.0], colors='red', linewidths=2, linestyles='--')
        ax1.plot(s_hat, lambda_hat, 'ro', markersize=10, label=f'Current: ({s_hat:.2f}, {lambda_hat:.2f})')
        ax1.set_xlabel('Smear ŝ')
        ax1.set_ylabel('Load λ̂')
        ax1.set_title(f'{mechanism_name}: DoF Fraction')
        ax1.legend()
        plt.colorbar(contour1, ax=ax1)
        
        # 2. Time dilation comparison
        ax2 = axes[0, 1]
        Gamma_standard = np.zeros_like(S)
        Gamma_dof = np.zeros_like(S)
        
        for i in range(len(s_range)):
            for j in range(len(l_range)):
                s, l = S[i,j], L[i,j]
                if substrate.capacity_constraint_satisfied(s, l):
                    Gamma_standard[i,j] = substrate.compute_gamma(s, l, p)
                    dof_factor = mechanism.compute_dof_reduction(substrate, s, l)
                    # Modified dilation including DoF effects
                    Gamma_dof[i,j] = Gamma_standard[i,j] / dof_factor
                else:
                    Gamma_standard[i,j] = np.nan
                    Gamma_dof[i,j] = np.nan
        
        contour2 = ax2.contourf(S, L, Gamma_dof - Gamma_standard, levels=20, cmap='RdBu_r')
        ax2.contour(S, L, S**2 + L**2, levels=[1.0], colors='black', linewidths=2, linestyles='--')
        ax2.plot(s_hat, lambda_hat, 'ko', markersize=10)
        ax2.set_xlabel('Smear ŝ')
        ax2.set_ylabel('Load λ̂')
        ax2.set_title('Dilation Difference (DoF - Standard)')
        plt.colorbar(contour2, ax=ax2)
        
        # 3. Current point analysis
        ax3 = axes[1, 0]
        if substrate.capacity_constraint_satisfied(s_hat, lambda_hat):
            dof_current = mechanism.compute_dof_reduction(substrate, s_hat, lambda_hat)
            gamma_standard = substrate.compute_gamma(s_hat, lambda_hat, p)
            gamma_dof = gamma_standard / dof_current
            
            metrics = ['DoF Fraction', 'Standard Γ', 'DoF-Modified Γ', 'Difference']
            values = [dof_current, gamma_standard, gamma_dof, gamma_dof - gamma_standard]
            colors = ['blue', 'green', 'red', 'orange']
            
            bars = ax3.bar(metrics, values, color=colors, alpha=0.7)
            ax3.set_ylabel('Value')
            ax3.set_title(f'Current Point Analysis\nŝ={s_hat:.2f}, λ̂={lambda_hat:.2f}')
            ax3.tick_params(axis='x', rotation=45)
            
            # Add value labels on bars
            for bar, value in zip(bars, values):
                height = bar.get_height()
                ax3.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                        f'{value:.3f}', ha='center', va='bottom')
        else:
            ax3.text(0.5, 0.5, 'Capacity constraint\nviolated!', 
                    ha='center', va='center', transform=ax3.transAxes,
                    fontsize=16, color='red')
            ax3.set_title('Invalid Parameter Region')
        
        # 4. Mechanism signature
        ax4 = axes[1, 1]
        ax4.axis('off')
        signature = mechanism.get_signature()
        signature_text = f"Mechanism: {mechanism_name}\n\n"
        for key, value in signature.items():
            signature_text += f"{key.replace('_', ' ').title()}: {value}\n\n"
        
        ax4.text(0.05, 0.95, signature_text, transform=ax4.transAxes,
                fontsize=10, verticalalignment='top', fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
        
        plt.tight_layout()
        plt.show()
    
    # Create interactive widget
    interactive_plot = widgets.interactive(
        update_plot,
        mechanism_name=mechanism_dropdown,
        s_hat=s_hat_slider,
        lambda_hat=lambda_hat_slider,
        p=p_slider
    )
    
    return interactive_plot

# Create and display the explorer
explorer = create_mechanism_explorer()
display(explorer)

## B(N) Derivation Attempt

Let's try to derive the constitutive relation B(N) from the DoF mechanisms instead of curve-fitting to Schwarzschild.

In [ ]:
def attempt_b_n_derivation(mechanism: DoFMechanism, substrate: SubstrateNetwork):
    """Attempt to derive B(N) from a DoF mechanism."""
    
    print(f"=== B(N) Derivation for {mechanism.__class__.__name__} ===")
    
    # Test range of N values (lapse function)
    N_values = np.linspace(0.1, 1.0, 20)
    lambda_hat_values = 1.0 - N_values  # λ̂ = 1 - N
    
    # Compute DoF reduction for pure curvature (no motion)
    s_hat = 0.0
    dof_fractions = []
    
    for lambda_hat in lambda_hat_values:
        if substrate.capacity_constraint_satisfied(s_hat, lambda_hat):
            dof_fraction = mechanism.compute_dof_reduction(substrate, s_hat, lambda_hat)
            dof_fractions.append(dof_fraction)
        else:
            dof_fractions.append(np.nan)
    
    dof_fractions = np.array(dof_fractions)
    
    # Attempt to derive B(N) from DoF behavior
    # Hypothesis: B(N) ~ f(DoF_fraction)
    
    # Try different functional forms
    forms = {
        'Linear': lambda dof: 1 + 3 * (1 - dof),
        'Quadratic': lambda dof: (2 - dof)**2,
        'Exponential': lambda dof: np.exp(2 * (1 - dof)),
        'Power Law': lambda dof: (1/dof)**2,
        'Schwarzschild': lambda N: (2/(1+N))**4  # Target for comparison
    }
    
    plt.figure(figsize=(15, 10))
    
    # Plot 1: DoF vs N
    plt.subplot(2, 3, 1)
    valid_mask = ~np.isnan(dof_fractions)
    plt.plot(N_values[valid_mask], dof_fractions[valid_mask], 'bo-', label='DoF Fraction')
    plt.xlabel('Lapse N')
    plt.ylabel('DoF Fraction')
    plt.title('DoF vs Lapse Function')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    # Plot 2-5: Different B(N) forms derived from DoF
    colors = ['red', 'green', 'purple', 'orange']
    form_names = ['Linear', 'Quadratic', 'Exponential', 'Power Law']
    
    for i, (name, color) in enumerate(zip(form_names, colors)):
        plt.subplot(2, 3, i+2)
        
        # Compute B(N) from DoF
        B_from_dof = forms[name](dof_fractions[valid_mask])
        
        # Compute target Schwarzschild B(N)
        B_schwarzschild = forms['Schwarzschild'](N_values[valid_mask])
        
        plt.plot(N_values[valid_mask], B_from_dof, color=color, linewidth=2, 
                label=f'{name} from DoF')
        plt.plot(N_values[valid_mask], B_schwarzschild, 'k--', linewidth=2, 
                label='Schwarzschild Target')
        
        plt.xlabel('Lapse N')
        plt.ylabel('B(N)')
        plt.title(f'B(N): {name} Form')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.yscale('log')
    
    # Plot 6: Comparison of all forms
    plt.subplot(2, 3, 6)
    B_schwarzschild = forms['Schwarzschild'](N_values[valid_mask])
    plt.plot(N_values[valid_mask], B_schwarzschild, 'k-', linewidth=3, 
            label='Schwarzschild Target')
    
    for name, color in zip(form_names, colors):
        B_from_dof = forms[name](dof_fractions[valid_mask])
        plt.plot(N_values[valid_mask], B_from_dof, color=color, linewidth=2, 
                alpha=0.7, label=f'{name}')
    
    plt.xlabel('Lapse N')
    plt.ylabel('B(N)')
    plt.title('All B(N) Forms Comparison')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.yscale('log')
    
    plt.tight_layout()
    plt.show()
    
    # Compute goodness of fit
    print(f'\nGoodness of fit to Schwarzschild B(N):')  
    for name, color in zip(form_names, colors):
        B_from_dof = forms[name](dof_fractions[valid_mask])
        mse = np.mean((B_from_dof - B_schwarzschild)**2)
        print(f'{name:12}: MSE = {mse:.6f}')
    
    return N_values, dof_fractions

# Test B(N) derivation for each mechanism
mechanisms = {
    'Causal Diamond': CausalDiamondThrottling(),
    'Tension Bandgaps': TensionInducedBandgaps(), 
    'Mode Crowding': ModeCrowdingRedshift()
}

substrate = SubstrateNetwork()

for name, mechanism in mechanisms.items():
    print(f'\n{"="*50}')
    print(f'Testing {name} mechanism')
    print(f'{"="*50}')
    N_vals, dof_vals = attempt_b_n_derivation(mechanism, substrate)

## Thermodynamic and String Theory Connections

Let's explore the connections to thermodynamics and string theory discussed in the GPT-5 conversation.

In [ ]:
def analyze_thermodynamic_connection(mechanism: DoFMechanism, substrate: SubstrateNetwork):
    """Analyze the thermodynamic interpretation of DoF mechanisms."""
    
    print(f"=== Thermodynamic Analysis: {mechanism.__class__.__name__} ===")
    
    # Test points in parameter space
    test_points = [
        (0.0, 0.0, "Rest frame"),
        (0.5, 0.0, "High velocity"),
        (0.0, 0.5, "Strong gravity"),
        (0.3, 0.4, "Combined effects")
    ]
    
    results = []
    
    for s_hat, lambda_hat, description in test_points:
        if substrate.capacity_constraint_satisfied(s_hat, lambda_hat):
            # Compute DoF and interpret as entropy
            dof_fraction = mechanism.compute_dof_reduction(substrate, s_hat, lambda_hat)
            
            # Entropy ~ log(DoF)
            entropy = np.log(max(dof_fraction, 1e-10))  # Avoid log(0)
            
            # Temperature ~ 1/entropy_gradient (simplified)
            # In full theory, this would be ∂S/∂E
            temperature = 1.0 / max(abs(entropy), 0.1)
            
            # Time dilation factors
            gamma_standard = substrate.compute_gamma(s_hat, lambda_hat)
            gamma_dof = gamma_standard / dof_fraction
            
            results.append({
                'description': description,
                's_hat': s_hat,
                'lambda_hat': lambda_hat,
                'dof_fraction': dof_fraction,
                'entropy': entropy,
                'temperature': temperature,
                'gamma_standard': gamma_standard,
                'gamma_dof': gamma_dof
            })
    
    # Create summary table
    print(f"\n{'Description':<15} {'ŝ':<5} {'λ̂':<5} {'DoF':<6} {'S':<8} {'T':<8} {'Γ_std':<8} {'Γ_DoF':<8}")
    print("-" * 70)
    
    for r in results:
        print(f"{r['description']:<15} {r['s_hat']:<5.1f} {r['lambda_hat']:<5.1f} "
              f"{r['dof_fraction']:<6.3f} {r['entropy']:<8.3f} {r['temperature']:<8.3f} "
              f"{r['gamma_standard']:<8.3f} {r['gamma_dof']:<8.3f}")
    
    # Plot thermodynamic relationships
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Extract data for plotting
    dof_fractions = [r['dof_fraction'] for r in results]
    entropies = [r['entropy'] for r in results]
    temperatures = [r['temperature'] for r in results]
    gamma_dofs = [r['gamma_dof'] for r in results]
    descriptions = [r['description'] for r in results]
    
    # Plot 1: DoF vs Entropy
    axes[0,0].scatter(dof_fractions, entropies, s=100, alpha=0.7)
    for i, desc in enumerate(descriptions):
        axes[0,0].annotate(desc, (dof_fractions[i], entropies[i]), 
                          xytext=(5, 5), textcoords='offset points')
    axes[0,0].set_xlabel('DoF Fraction')
    axes[0,0].set_ylabel('Entropy S = log(DoF)')
    axes[0,0].set_title('DoF-Entropy Relationship')
    axes[0,0].grid(True, alpha=0.3)
    
    # Plot 2: Temperature vs Time Dilation
    axes[0,1].scatter(temperatures, gamma_dofs, s=100, alpha=0.7, color='red')
    for i, desc in enumerate(descriptions):
        axes[0,1].annotate(desc, (temperatures[i], gamma_dofs[i]), 
                          xytext=(5, 5), textcoords='offset points')
    axes[0,1].set_xlabel('Effective Temperature')
    axes[0,1].set_ylabel('Time Dilation Γ')
    axes[0,1].set_title('Temperature-Dilation Relationship')
    axes[0,1].grid(True, alpha=0.3)
    
    # Plot 3: Entropy vs Time Dilation
    axes[1,0].scatter(entropies, gamma_dofs, s=100, alpha=0.7, color='green')
    for i, desc in enumerate(descriptions):
        axes[1,0].annotate(desc, (entropies[i], gamma_dofs[i]), 
                          xytext=(5, 5), textcoords='offset points')
    axes[1,0].set_xlabel('Entropy S')
    axes[1,0].set_ylabel('Time Dilation Γ')
    axes[1,0].set_title('Entropy-Dilation Relationship')
    axes[1,0].grid(True, alpha=0.3)
    
    # Plot 4: Phase diagram
    s_hats = [r['s_hat'] for r in results]
    lambda_hats = [r['lambda_hat'] for r in results]
    
    scatter = axes[1,1].scatter(s_hats, lambda_hats, c=entropies, s=200, 
                               cmap='viridis', alpha=0.8)
    for i, desc in enumerate(descriptions):
        axes[1,1].annotate(desc, (s_hats[i], lambda_hats[i]), 
                          xytext=(5, 5), textcoords='offset points', color='white')
    
    # Add capacity constraint circle
    theta = np.linspace(0, 2*np.pi, 100)
    axes[1,1].plot(np.cos(theta), np.sin(theta), 'r--', linewidth=2, 
                  label='Capacity constraint')
    
    axes[1,1].set_xlabel('Smear ŝ')
    axes[1,1].set_ylabel('Load λ̂')
    axes[1,1].set_title('Entropy Phase Diagram')
    axes[1,1].legend()
    plt.colorbar(scatter, ax=axes[1,1], label='Entropy S')
    
    plt.tight_layout()
    plt.show()
    
    return results

# Analyze thermodynamic connections for each mechanism
for name, mechanism in mechanisms.items():
    print(f'\n{"="*60}')
    print(f'Thermodynamic analysis: {name}')
    print(f'{"="*60}')
    thermo_results = analyze_thermodynamic_connection(mechanism, substrate)

## Conclusions and Next Steps

This notebook has explored different mechanisms for how degrees of freedom change with motion and curvature in a substrate network. Key findings:

### Mechanism Comparison
- **Causal Diamond Throttling**: Provides smooth, volume-based DoF reduction
- **Tension-Induced Bandgaps**: Creates frequency-dependent effects and dispersion
- **Mode Crowding**: Systematic redshift effects on accessible frequencies

### B(N) Derivation
The attempts to derive B(N) from first principles show which functional forms naturally emerge from each mechanism. This is crucial for moving beyond curve-fitting.

### Thermodynamic Connections
The entropy interpretation (S ~ log(DoF)) provides a natural bridge to statistical mechanics and suggests that time dilation is fundamentally about information processing capacity.

### Future Work
1. Implement remaining mechanisms (entanglement compression, connectivity pruning, etc.)
2. Test against observational data (gravitational wave dispersion, etc.)
3. Develop string theory connections more rigorously
4. Create falsifiable predictions that differ from GR

### Google Colab Usage
This notebook is designed to run in Google Colab. Simply:
1. Click the "Open in Colab" badge at the top
2. Run all cells to explore the mechanisms interactively
3. Modify parameters to test your own hypotheses
4. Export results for further analysis

The interactive widgets allow real-time exploration of the parameter space and comparison between different mechanisms.